In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
          CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_store
          --PARTITION BY end_date CLUSTER BY store_id AS
WITH 
temp AS (
    SELECT * EXCEPT(rn) 
    FROM (
        SELECT 
            a.site store_id,
            a.assortment,
            a.date_from start_date,
            a.date_to end_date,
            a.is_delete,
            0 block_by_site,
            IF(c.status = 'ACTIVE', 0, 1) block_by_assortment,
            IF(a.assortment NOT LIKE 'EX_%', 0, 1) block_by_ex,
            ROW_NUMBER()  OVER (PARTITION BY a.site, a.assortment ORDER BY aedat DESC) rn 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_assortment_site a
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_desc c ON a.assortment = c.assortment
        UNION ALL
        SELECT 
            a.locnr store_id,
            a.asort assortment,
            a.datab start_date,
            a.datbi end_date,
            0 is_delete,
            0 block_by_site,
            IF(c.status = 'ACTIVE', 0, 1) block_by_assortment,
            IF(a.asort NOT LIKE 'EX_%', 0, 1) block_by_ex,
            ROW_NUMBER()  OVER (PARTITION BY a.locnr, a.asort ORDER BY reqtsn DESC, datapakid DESC, record DESC) rn 
        FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_vccmdm40 a
            LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_desc c ON a.asort = c.assortment
    ) 
    WHERE rn = 1
)
SELECT * FROM temp;
          """)